*Генерация тестовых данных по продажам для дашборда*

![](model.png)

    Наша задача заполнить тестовыми данными эту модель, чтобы были данные для начала разработки дашборда

Создаем БД и нашу модель в Sqlite. Нужно создать и заполнить следующие таблицы:

- DIM_PRODUCTS
- DIM_MANAGERS
- DIM_MANAGERS_KPI
- DIM_BRANCHES
- DIM_BRANCHES_GEO
- FCT_SALES

Здесь префикс DIM это dimension tables, FCT fact table.

БД сохраняется в каталоге этого notebook в файле **test_db.sqlite**

In [1]:
from sqlalchemy import create_engine
import os

db_name = 'test_db.sqlite'

try:
    os.remove('./'+db_name)
except:
    if not connection.closed:
        connection.close()
        os.remove('./'+db_name)

engine = create_engine('sqlite:///'+db_name)
connection = engine.connect()

In [2]:
from sqlalchemy import MetaData, Table, Column, String, Integer, Date, Numeric
from datetime import datetime

metadata = MetaData()
       
managers = Table('DIM_MANAGERS', metadata, 
                     Column('ID', Integer()), 
                     Column('EMPLOYEE_CODE', String(50)),
                     Column('FULL_NAME', String(250)),
                     Column('DT_VALID_FROM', Date(), default=datetime.strptime('01-01-2018', '%d-%m-%Y')),
                     Column('DT_VALID_TO', Date(), default=datetime.strptime('31-12-9999', '%d-%m-%Y'))
                    )
    
products = Table('DIM_PRODUCTS', metadata,
                    Column('ID', Integer()),
                    Column('CODE', String(50)),
                    Column('NAME', String(250)),
                    Column('DT_VALID_FROM', Date(), default=datetime.strptime('01-01-2018', '%d-%m-%Y')),
                    Column('DT_VALID_TO', Date(), default=datetime.strptime('31-12-9999', '%d-%m-%Y'))
                   )
    
branches = Table('DIM_BRANCHES', metadata,
                    Column('ID', Integer()),
                    Column('CODE', String(50)),
                    Column('NAME', String(250)),
                    Column('DT_VALID_FROM', Date(), default=datetime.strptime('01-01-2018', '%d-%m-%Y')),
                    Column('DT_VALID_TO', Date(), default=datetime.strptime('31-12-9999', '%d-%m-%Y'))
                   )
    
managers_kpi = Table('DIM_MANAGERS_KPI', metadata, 
                     Column('MANAGER_ID', Integer()), 
                     Column('KPI', Numeric(5,2)),
                     Column('DT_VALID_FROM', Date(), default=datetime.strptime('01-01-2018', '%d-%m-%Y')),
                     Column('DT_VALID_TO', Date(), default=datetime.strptime('31-12-9999', '%d-%m-%Y'))
                    )
    
branches_geo = Table('DIM_BRANCHES_GEO', metadata,
                    Column('BRANCH_ID', Integer()),
                    Column('CITY', String(50)),
                    Column('LONGITUDE', Numeric(5,9)),
                    Column('LATITUDE', Numeric(5,9)),
                    Column('ADDRESS', String(250)),
                    Column('REGION_ISO_CODE', String(30)),
                    Column('DT_VALID_FROM', Date(), default=datetime.strptime('01-01-2018', '%d-%m-%Y')),
                    Column('DT_VALID_TO', Date(), default=datetime.strptime('31-12-9999', '%d-%m-%Y'))
                   )
    
sales = Table('FCT_SALES', metadata,
                  Column('OPERATION_DATE', Date()),
                  Column('PRODUCT_ID', Integer()),
                  Column('MANAGER_ID', Integer()),
                  Column('BRANCH_ID', Integer()),
                  Column('QUANTITY', Integer()),  
                  Column('AMOUNT', Numeric(18,2)),
                  Column('AMOUNT_KZT', Numeric(18,2))
                  )
metadata.create_all(engine)
engine.table_names()

['DIM_BRANCHES',
 'DIM_BRANCHES_GEO',
 'DIM_MANAGERS',
 'DIM_MANAGERS_KPI',
 'DIM_PRODUCTS',
 'FCT_SALES']

Скачиваем датасет с ФИО людей и распаковываем

In [4]:
from urllib.request import urlretrieve

url = 'https://mydata.biz/storage/download/ebcdfe6fb2d546398010e0d6564a79bb/%D0%91%D0%B0%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B8%D0%BC%D0%B5%D0%BD%20%D0%B8%20%D1%84%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D0%B9%20%D0%B2%20%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B5%20CSV.zip'
file_name = 'human_names.zip'
urlretrieve(url, file_name)

('human_names.zip', <http.client.HTTPMessage at 0x2a383724dd8>)

In [5]:
import zipfile
arc = zipfile.ZipFile(file_name, 'r')
dir_name = './data/human_names'
arc.extractall(dir_name)
arc.close()

print(os.listdir(dir_name))

['foreign_names.csv', 'russian_names.csv', 'russian_surnames.csv', '_readme.txt']


Генерим менеджеров, и заполняем DIM_MANAGERS

In [6]:
import pandas as pd

f_nm = pd.read_csv(dir_name + '/russian_names.csv', sep=';', usecols=['Name'], squeeze=True).dropna()
l_nm = pd.read_csv(dir_name + '/russian_surnames.csv', sep=';', usecols=['Surname'], squeeze=True).dropna()

print(len(f_nm))
print(len(l_nm))

print(f_nm[:3])
print(l_nm[:3])

51529
318474
0       Аалия
1    Аанжелла
2         Аба
Name: Name, dtype: object
0     Аалферов
1    Аалферова
2         Ааль
Name: Surname, dtype: object


- Выбираем случайные Фамилию и Имя из скачанного датасета
- генерим айди

In [7]:
import numpy as np

n_mgr = 1500
ids = [x+1001 for x in range(n_mgr)]
print(ids[:10])

inds_f = np.random.choice(np.arange(len(f_nm)), size = n_mgr)
inds_l = np.random.choice(np.arange(len(l_nm)), size = n_mgr)

f_nm = f_nm.iloc[inds_f]
l_nm = l_nm.iloc[inds_l]

values = [{'ID' : k1, 'EMPLOYEE_CODE' : k2, 'FULL_NAME' : fn + ' ' + ln} for (k1, k2, fn, ln) in zip(ids, 
                                                                                 ['UMEN_'+str(x) for x in ids], 
                                                                                 f_nm, l_nm)]

values[:5]

[1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]


[{'EMPLOYEE_CODE': 'UMEN_1001', 'FULL_NAME': 'Мурод Жебраков', 'ID': 1001},
 {'EMPLOYEE_CODE': 'UMEN_1002', 'FULL_NAME': 'Бафтазер Баткеева', 'ID': 1002},
 {'EMPLOYEE_CODE': 'UMEN_1003', 'FULL_NAME': 'Чар Богоявленский', 'ID': 1003},
 {'EMPLOYEE_CODE': 'UMEN_1004',
  'FULL_NAME': 'Исламбек Балаченков',
  'ID': 1004},
 {'EMPLOYEE_CODE': 'UMEN_1005', 'FULL_NAME': 'Борис Самосенко', 'ID': 1005}]

In [8]:
from sqlalchemy import insert

rs = connection.execute(insert(managers), values)
print(rs.rowcount)

1500


Заполняем DIM_MANAGERS_KPI

In [9]:
from sqlalchemy import select
rs = connection.execute(select([managers.columns.ID])).fetchall()

values = [{'MANAGER_ID' : x[0], 'KPI' : round(np.random.random()*10, 2)} for x in rs]
print(values[:5])

rs = connection.execute(insert(managers_kpi), values)
print(rs.rowcount)

[{'MANAGER_ID': 1001, 'KPI': 6.02}, {'MANAGER_ID': 1002, 'KPI': 0.86}, {'MANAGER_ID': 1003, 'KPI': 8.72}, {'MANAGER_ID': 1004, 'KPI': 4.7}, {'MANAGER_ID': 1005, 'KPI': 7.01}]
1500


Заполняем DIM_PRODUCTS

In [10]:
ids = [x for x in range(100)]
product_types = ['DEPOSIT', 'LOAN', 'MORTGAGE', 'CARD', 'AUTOCRED', 'TRANSFER']
product_types_n = ['Депозит', 'Кредит', 'Ипотека', 'Карта', 'Автокредит', 'Перевод']

values = [{'ID':i+1, 
           'CODE':product_types[j]+'_'+str(i), 
           'NAME':product_types_n[j]+'_'+str(i)} for i, j in zip(ids,
                                                                 [x for x in np.random.choice(len(product_types), size=100)])]

print(values[:5])

rs = connection.execute(insert(products), values)
print(rs.rowcount)

[{'NAME': 'Автокредит_0', 'CODE': 'AUTOCRED_0', 'ID': 1}, {'NAME': 'Карта_1', 'CODE': 'CARD_1', 'ID': 2}, {'NAME': 'Ипотека_2', 'CODE': 'MORTGAGE_2', 'ID': 3}, {'NAME': 'Кредит_3', 'CODE': 'LOAN_3', 'ID': 4}, {'NAME': 'Карта_4', 'CODE': 'CARD_4', 'ID': 5}]
100


Заполняем DIM_BRANCHES и DIM_BRANCHES_GEO из живых данных, для визуализации на реальных геоданных

Скрейпим сайт eubank.kz

In [11]:
import requests, re

url = 'https://www.eubank.kz/map/filial/'
html = requests.get(url).text

from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

city = [{'city' : x.text, 'id': x['value']} for x in soup.find('select', {'class':'big_select'}).find_all('option')]
print(city)

[{'city': 'Алматы', 'id': '1'}, {'city': 'Астана', 'id': '3'}, {'city': 'Актау', 'id': '6'}, {'city': 'Актобе', 'id': '11'}, {'city': 'Атырау', 'id': '18'}, {'city': 'Караганда', 'id': '5'}, {'city': 'Кокшетау', 'id': '37'}, {'city': 'Костанай', 'id': '4'}, {'city': 'Кызылорда', 'id': '41'}, {'city': 'Павлодар', 'id': '23'}, {'city': 'Петропавловск', 'id': '26'}, {'city': 'Талдыкорган', 'id': '31'}, {'city': 'Тараз', 'id': '20'}, {'city': 'Уральск', 'id': '24'}, {'city': 'Усть-Каменогорск', 'id': '19'}, {'city': 'Шымкент', 'id': '27'}]


In [12]:
branches_from_web = []
for r in city:
    html = requests.get(url+'?city='+r['id']).text
    soup = BeautifulSoup(html, "html.parser")
    for rows in soup.find('table', {'class':'list_table'}).find_all('tr'):
        if rows.find('td'):
            branches_from_web.append({'city' : r['city'],
                                  'name' : rows.find('td', {'class':'name_bank_inner'}).text,
                                  'address' : rows.find('a', {'class':'yandex-a 1'}).text,
                                  'lon' : re.search(r'Map\(.*?(\d+\.\d+).*?(\d+\.\d+)',
                                                    rows.find('a', {'class':'yandex-a 1'})['href']).group(1),
                                  'lat' : re.search(r'Map\(.*?(\d+\.\d+).*?(\d+\.\d+)',
                                                    rows.find('a', {'class':'yandex-a 1'})['href']).group(2)
                                 })
        
print(len(branches_from_web))
print(branches_from_web[:5])

132
[{'lon': '76.9499533', 'city': 'Алматы', 'address': '050011, ул.Шолохова, 17/7', 'name': 'Отделение №601', 'lat': '43.335679'}, {'lon': '76.898144', 'city': 'Алматы', 'address': '050008, ул.Толе би, 187 "A", уг.ул.Жарокова', 'name': 'Отделение №605', 'lat': '43.252142'}, {'lon': '76.930958', 'city': 'Алматы', 'address': '050014, мкр.Айнабулак, ул.Жумабаева, 98 "A", уг.ул.Палладина', 'name': 'Отделение №607', 'lat': '43.317189'}, {'lon': '76.839916', 'city': 'Алматы', 'address': '050063, мкр.Жетысу 1, 55, пр.Абая, уг.ул.Б.Момышулы', 'name': 'Отделение №608', 'lat': '43.217485'}, {'lon': '76.945213', 'city': 'Алматы', 'address': '050013, пр.Абая, 12/38 "А"', 'name': 'Отделение №609', 'lat': '43.241707'}]


In [13]:
region_iso_kz = {'Алматы' : 'KZ-ALA', 
 'Астана' : 'KZ-AST',
 'Байконур' : 'KZ-BAY',
 'Талдыкорган' : 'KZ-ALM', 
 'Кокшетау' : 'KZ-AKM',
 'Актобе' : 'KZ-AKT',
 'Атырау' : 'KZ-ATY',
 'Усть-Каменогорск' : 'KZ-VOS',
 'Тараз' : 'KZ-ZHA',
 'Уральск' : 'KZ-ZAP',
 'Караганда' : 'KZ-KAR',
 'Костанай' : 'KZ-KUS',
 'Кызылорда' : 'KZ-KZY',
 'Актау' : 'KZ-MAN',
 'Павлодар' : 'KZ-PAV',
 'Петропавловск' : 'KZ-SEV',
 'Шымкент' : 'KZ-YUZ'}

values = [{'ID': k+1, 
           'CODE': re.search(r'№(\d+)', v['name']).group(1), 
           'NAME': v['name']} for k, v in enumerate(branches_from_web)]
print(values[:10])

r = connection.execute(insert(branches), values)
print(r.rowcount)

[{'NAME': 'Отделение №601', 'CODE': '601', 'ID': 1}, {'NAME': 'Отделение №605', 'CODE': '605', 'ID': 2}, {'NAME': 'Отделение №607', 'CODE': '607', 'ID': 3}, {'NAME': 'Отделение №608', 'CODE': '608', 'ID': 4}, {'NAME': 'Отделение №609', 'CODE': '609', 'ID': 5}, {'NAME': 'Отделение №610', 'CODE': '610', 'ID': 6}, {'NAME': 'Отделение №613 ', 'CODE': '613', 'ID': 7}, {'NAME': 'Отделение №617', 'CODE': '617', 'ID': 8}, {'NAME': 'Отделение №620', 'CODE': '620', 'ID': 9}, {'NAME': 'Отделение №621', 'CODE': '621', 'ID': 10}]
132


In [14]:
values = [{'BRANCH_ID': k+1, 
           'CITY': v['city'], 
           'LONGITUDE': v['lon'],
           'LATITUDE' : v['lat'],
           'ADDRESS' : v['address'],
           'REGION_ISO_CODE' : region_iso_kz[v['city']] } for k, v in enumerate(branches_from_web)]
print(values[:10])

r = connection.execute(insert(branches_geo), values)
print(r.rowcount)

[{'LONGITUDE': '76.9499533', 'CITY': 'Алматы', 'BRANCH_ID': 1, 'ADDRESS': '050011, ул.Шолохова, 17/7', 'LATITUDE': '43.335679', 'REGION_ISO_CODE': 'KZ-ALA'}, {'LONGITUDE': '76.898144', 'CITY': 'Алматы', 'BRANCH_ID': 2, 'ADDRESS': '050008, ул.Толе би, 187 "A", уг.ул.Жарокова', 'LATITUDE': '43.252142', 'REGION_ISO_CODE': 'KZ-ALA'}, {'LONGITUDE': '76.930958', 'CITY': 'Алматы', 'BRANCH_ID': 3, 'ADDRESS': '050014, мкр.Айнабулак, ул.Жумабаева, 98 "A", уг.ул.Палладина', 'LATITUDE': '43.317189', 'REGION_ISO_CODE': 'KZ-ALA'}, {'LONGITUDE': '76.839916', 'CITY': 'Алматы', 'BRANCH_ID': 4, 'ADDRESS': '050063, мкр.Жетысу 1, 55, пр.Абая, уг.ул.Б.Момышулы', 'LATITUDE': '43.217485', 'REGION_ISO_CODE': 'KZ-ALA'}, {'LONGITUDE': '76.945213', 'CITY': 'Алматы', 'BRANCH_ID': 5, 'ADDRESS': '050013, пр.Абая, 12/38 "А"', 'LATITUDE': '43.241707', 'REGION_ISO_CODE': 'KZ-ALA'}, {'LONGITUDE': '76.904469', 'CITY': 'Алматы', 'BRANCH_ID': 6, 'ADDRESS': '050043, мкр.Мирас, 188', 'LATITUDE': '43.132231', 'REGION_ISO_COD

Заполняем FCT_SALES

In [15]:
#from sqlalchemy import func

rs_mgr = [x[0] for x in connection.execute(select([managers.columns.ID])).fetchall()]
rs_prod = [x[0] for x in connection.execute(select([products.columns.ID])).fetchall()]
rs_brn = [x[0] for x in connection.execute(select([branches.columns.ID])).fetchall()]

rs_mgr = [(x, rs_brn[np.random.choice(len(rs_brn), size=1).tolist()[0]]) for x in rs_mgr]

today = datetime.now().strftime('%d-%m-%Y')
dates = [x for x in pd.date_range('01-01-2018', today).to_pydatetime().tolist() if x.weekday() not in [5,6]]

In [16]:
values = []
for d in dates:
    for mgr in rs_mgr:
        p = np.random.random()
        if round(p*10) % 2 == 0:
            for pr in rs_prod:
                p = np.random.random()
                if p > 0.6:
                    p = round(np.random.random() * 20)
                    if p>0:
                        values.append({'OPERATION_DATE' : d,
                                       'PRODUCT_ID' : pr,
                                       'MANAGER_ID' : mgr[0],
                                       'BRANCH_ID' : mgr[1],
                                       'QUANTITY' : p
                                      })

In [ ]:
values[:3]

st = 0
for i in range(0, len(values)+1, 1000):
    r = connection.execute(insert(sales), values[st:i])
    st = i
    
#print(r.rowcount)

Собираю витрину

In [21]:
from sqlalchemy import and_, func

dm_query = select([managers.columns.EMPLOYEE_CODE,
            managers.c.FULL_NAME.label('MANAGER_NAME'),
            products.c.CODE.label('PRODUCT_CODE'),
            products.c.NAME.label('PRODUCT_NAME'),
            branches.c.CODE.label('BRANCH_CODE'),
            branches.c.NAME.label('BRANCH_NAME'),
            branches_geo.c.LONGITUDE.label('BRANCH_LON'),
            branches_geo.c.LATITUDE.label('BRANCH_LAT'),
            branches_geo.c.REGION_ISO_CODE.label('REGION_ISO_CODE'),
            managers_kpi.c.KPI,
            sales.c.OPERATION_DATE,            
            func.sum(sales.c.QUANTITY).label('QUANTITY')
           ]).where(and_(sales.c.OPERATION_DATE.between(managers.c.DT_VALID_FROM,managers.c.DT_VALID_TO),\
                         sales.c.OPERATION_DATE.between(products.c.DT_VALID_FROM,products.c.DT_VALID_TO),\
                         sales.c.OPERATION_DATE.between(branches.c.DT_VALID_FROM,branches.c.DT_VALID_TO),\
                         sales.c.OPERATION_DATE.between(branches_geo.c.DT_VALID_FROM,branches_geo.c.DT_VALID_TO),\
                         sales.c.OPERATION_DATE.between(managers_kpi.c.DT_VALID_FROM,managers_kpi.c.DT_VALID_TO)\
                        ))\
    .where(and_(sales.c.MANAGER_ID==managers.c.ID, sales.c.BRANCH_ID==branches.c.ID,\
                sales.c.PRODUCT_ID==products.c.ID, managers_kpi.c.MANAGER_ID==managers.c.ID,\
                branches_geo.c.BRANCH_ID==branches.c.ID))\
    .group_by(managers.columns.EMPLOYEE_CODE,
              managers.c.FULL_NAME,
              products.c.CODE,
              products.c.NAME,
              branches.c.CODE,
              branches.c.NAME,
              branches_geo.c.LONGITUDE,
              branches_geo.c.LATITUDE,
              branches_geo.c.REGION_ISO_CODE,
              managers_kpi.c.KPI,
              sales.c.OPERATION_DATE)\
    .order_by(sales.c.OPERATION_DATE, branches.c.CODE)

str(dm_query)

'SELECT "DIM_MANAGERS"."EMPLOYEE_CODE", "DIM_MANAGERS"."FULL_NAME" AS "MANAGER_NAME", "DIM_PRODUCTS"."CODE" AS "PRODUCT_CODE", "DIM_PRODUCTS"."NAME" AS "PRODUCT_NAME", "DIM_BRANCHES"."CODE" AS "BRANCH_CODE", "DIM_BRANCHES"."NAME" AS "BRANCH_NAME", "DIM_BRANCHES_GEO"."LONGITUDE" AS "BRANCH_LON", "DIM_BRANCHES_GEO"."LATITUDE" AS "BRANCH_LAT", "DIM_BRANCHES_GEO"."REGION_ISO_CODE" AS "REGION_ISO_CODE", "DIM_MANAGERS_KPI"."KPI", "FCT_SALES"."OPERATION_DATE", sum("FCT_SALES"."QUANTITY") AS "QUANTITY" \nFROM "DIM_MANAGERS", "DIM_PRODUCTS", "DIM_BRANCHES", "DIM_BRANCHES_GEO", "DIM_MANAGERS_KPI", "FCT_SALES" \nWHERE "FCT_SALES"."OPERATION_DATE" BETWEEN "DIM_MANAGERS"."DT_VALID_FROM" AND "DIM_MANAGERS"."DT_VALID_TO" AND "FCT_SALES"."OPERATION_DATE" BETWEEN "DIM_PRODUCTS"."DT_VALID_FROM" AND "DIM_PRODUCTS"."DT_VALID_TO" AND "FCT_SALES"."OPERATION_DATE" BETWEEN "DIM_BRANCHES"."DT_VALID_FROM" AND "DIM_BRANCHES"."DT_VALID_TO" AND "FCT_SALES"."OPERATION_DATE" BETWEEN "DIM_BRANCHES_GEO"."DT_VALID_FROM

In [ ]:
rp = connection.execute(dm_query)
more_rows = True
i=0

while more_rows:
    partial_results = rp.fetchmany(500000)    
    if partial_results == []:
        more_rows = False
    else:
        df = pd.DataFrame(partial_results)   
        df.columns = partial_results[0].keys()
        df.to_csv('./DM_SALES'+str(i)+'.csv', sep=';', encoding='utf-8')
        i+=1

C:\Users\a.iskaliyev\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sqlalchemy\sql\sqltypes.py:596: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [25]:
connection.close()